In [1]:
import docker
import yaml

def get_running_models(client, services):
    containers = client.containers.list()
    models = [container for container in containers if container.name in services]
    return models

def get_internal_ports(container):
    ports = container.attrs['NetworkSettings']['Ports']
    internal_ports = [port.split("/")[0] for port in ports.keys() if ports[port]]
    return internal_ports

def get_compose_services(compose_file):
    with open(compose_file, 'r') as file:
        compose_data = yaml.safe_load(file)
    
    services = compose_data.get('services', {})
    speech2text_services = [service_name for service_name in services if service_name.startswith('speech2text')]
    
    return speech2text_services


client = docker.from_env()

compose_file_path = '../docker-compose.yml'
speech2text_services = get_compose_services(compose_file_path)
print("Сервисы из docker-compose.yml:")
for service in speech2text_services:
    print(f"Сервис: {service}")

running_models = get_running_models(client, speech2text_services)
print("\nЗапущенные модели:")

for model in running_models:
    internal_ports = get_internal_ports(model)
    if internal_ports:
        for internal_port in internal_ports:
            url = f"http://{model.name}:{internal_port}/"
            print(f"Сервис: {model.name}, URL: {url}")
    else:
        print(f"Сервис: {model.name}, Внутренние порты не найдены")



Сервисы из docker-compose.yml:
Сервис: speech2text-whisper-small

Запущенные модели:


In [12]:
client.version()

{'Platform': {'Name': 'Docker Engine - Community'},
 'Components': [{'Name': 'Engine',
   'Version': '26.1.4',
   'Details': {'ApiVersion': '1.45',
    'Arch': 'amd64',
    'BuildTime': '2024-06-05T11:28:55.000000000+00:00',
    'Experimental': 'false',
    'GitCommit': 'de5c9cf',
    'GoVersion': 'go1.21.11',
    'KernelVersion': '6.5.0-41-generic',
    'MinAPIVersion': '1.24',
    'Os': 'linux'}},
  {'Name': 'containerd',
   'Version': '1.6.33',
   'Details': {'GitCommit': 'd2d58213f83a351ca8f528a95fbd145f5654e957'}},
  {'Name': 'runc',
   'Version': '1.1.12',
   'Details': {'GitCommit': 'v1.1.12-0-g51d5e94'}},
  {'Name': 'docker-init',
   'Version': '0.19.0',
   'Details': {'GitCommit': 'de40ad0'}}],
 'Version': '26.1.4',
 'ApiVersion': '1.45',
 'MinAPIVersion': '1.24',
 'GitCommit': 'de5c9cf',
 'GoVersion': 'go1.21.11',
 'Os': 'linux',
 'Arch': 'amd64',
 'KernelVersion': '6.5.0-41-generic',
 'BuildTime': '2024-06-05T11:28:55.000000000+00:00'}

In [19]:
container = client.containers.get("06de068042bd")

In [26]:
for log in container.logs(stream=True):
    print(log)

b"INFO:     Will watch for changes in these directories: ['/app/src']\n"
b'INFO:     Uvicorn running on https://0.0.0.0:8000 (Press CTRL+C to quit)\n'
b'INFO:     Started reloader process [1] using WatchFiles\n'
b'Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.\n'
b"/usr/local/lib/python3.10/site-packages/gradio/interface.py:382: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.\n"
b'  warnings.warn(\n'
b'2024-06-19 09:36:16,730 - INFO - Running speech2text models.\n'
b'INFO:     Started server process [8]\n'
b'INFO:     Waiting for application startup.\n'
b'INFO:     Application startup complete.\n'
b'2024-06-19 09:36:16,866 - INFO - HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "\n'
b'2024-06-19 09:36:16,916 - INFO - HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "\n'
b'202

KeyboardInterrupt: 

In [28]:
for i in container.stats(stream=True):
    print(i)

b'{"read":"2024-06-19T10:45:00.641555547Z","preread":"0001-01-01T00:00:00Z","pids_stats":{"current":25,"limit":47889},"blkio_stats":{"io_service_bytes_recursive":[{"major":259,"minor":2,"op":"read","value":381169664},{"major":259,"minor":2,"op":"write","value":972136448}],"io_serviced_recursive":null,"io_queue_recursive":null,"io_service_time_recursive":null,"io_wait_time_recursive":null,"io_merged_recursive":null,"io_time_recursive":null,"sectors_recursive":null},"num_procs":0,"storage_stats":{},"cpu_stats":{"cpu_usage":{"total_usage":35581710000,"usage_in_kernelmode":10734302000,"usage_in_usermode":24847407000},"system_cpu_usage":43551470000000,"online_cpus":8,"throttling_data":{"periods":0,"throttled_periods":0,"throttled_time":0}},"precpu_stats":{"cpu_usage":{"total_usage":0,"usage_in_kernelmode":0,"usage_in_usermode":0},"throttling_data":{"periods":0,"throttled_periods":0,"throttled_time":0}},"memory_stats":{"usage":2072231936,"stats":{"active_anon":658452480,"active_file":2990039

KeyboardInterrupt: 

In [29]:
container.top()

{'Processes': [['root',
   '7564',
   '7543',
   '0',
   '12:34',
   '?',
   '00:00:02',
   '/usr/local/bin/python3 /usr/local/bin/uvicorn api.main:app --host 0.0.0.0 --port 8000 --reload --ssl-keyfile ssl/local_key.pem --ssl-certfile ssl/local_cert.pem'],
  ['root',
   '7602',
   '7564',
   '0',
   '12:34',
   '?',
   '00:00:00',
   '/usr/local/bin/python3 -c from multiprocessing.resource_tracker import main;main(4)'],
  ['root',
   '7603',
   '7564',
   '0',
   '12:34',
   '?',
   '00:00:33',
   '/usr/local/bin/python3 -c from multiprocessing.spawn import spawn_main; spawn_main(tracker_fd=5, pipe_handle=7) --multiprocessing-fork']],
 'Titles': ['UID', 'PID', 'PPID', 'C', 'STIME', 'TTY', 'TIME', 'CMD']}

In [4]:
import yaml

docker_compose_file = """
version: '3.8'

services:
  backend:
    image: dimkablin/s2t-factory-backend
    environment:
      - SSL_KEY=/app/ssl/local_key.pem
      - SSL_CERT=/app/ssl/local_cert.pem
    volumes:
      - ./backend/ssl:/app/ssl:ro
      - ./backend/.env:/app/.env:ro
      - /var/run/docker.sock:/var/run/docker.sock
    ports:
      - "8000:8000"
    networks:
      - app-network

  frontend:
    image: dimkablin/s2t-factory-frontend
    environment:
      - SSL_KEY=/app/ssl/local_key.pem
      - SSL_CERT=/app/ssl/local_cert.pem
    volumes:
      - ./frontend/ssl:/app/ssl:ro
      - ./frontend/.env:/app/.env:ro
    ports:
      - "8001:8001"
    networks:
      - app-network
    depends_on:
      - backend
    
  speech2text-whisper-small:
    image: dimkablin/speech2text
    environment:
      - MODEL_NAME=openai/whisper-small
      - USE_CUDA=True
      - PORT=8002
  
  speech2text-whisper-medium:
    image: dimkablin/speech2text
    environment:
      - MODEL_NAME=openai/whisper-medium
      - USE_CUDA=True
      - PORT=8003
"""

def get_speech2text_urls(file_path):
    with open(file_path, 'r') as file:
      yaml_content = file.read()

    data = yaml.safe_load(yaml_content)
    services = data['services']
    urls = {}

    for service_name, service_data in services.items():
        if 'speech2text' in service_name:
            port = None
            if 'ports' in service_data:
                port = service_data['ports'][0].split(':')[0]
            elif 'environment' in service_data:
                for env_var in service_data['environment']:
                    if env_var.startswith('PORT='):
                        port = env_var.split('=')[1]
                        break
            
            if port:
                urls[service_name] = port
    
    return urls

urls = get_speech2text_urls("../../docker-compose.yml")
urls

{'speech2text-whisper-small': '8002', 'speech2text-whisper-medium': '8003'}

In [5]:
list(urls.keys())

['speech2text-whisper-small', 'speech2text-whisper-medium']